<a href="https://colab.research.google.com/github/ericyang125/openpiv-python-gpu/blob/master/Openpiv_Python_Tutorial_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Openpiv-python-gpu Advanced Tutorial


Use the following link to run this using GPUs from Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericyang125/openpiv-python-gpu/blob/master/Openpiv_Python_Tutorial_Basic.ipynb)

Ensure that GPU acceleration is enabled in Google Colab: Runtime > Change runtime type.


## Introduction


This tutorial will show interactively how each of the parameters of the GPU function affects the output. The speed at which the GPU-accelerated algorithmn processes a large dataset is also demonstrated. Using GPU-acceleration can speed up processing by O(10):
> Dallas, C., Wu, M., Chou, V., Liberzon, A., & Sullivan, P. E. (2019). Graphical Processing Unit-Accelerated Open-Source Particle Image Velocimetry Software for High Performance Computing Systems. Journal of Fluids Engineering, 141(11).

The data used here are synthetic images generated from a slice of the Johns Hopkins turbulent channel database:
> Perlman, E., Burns, R., Li, Y., and Meneveau, C., 2007, “Data Exploration of Turbulence Simulations Using a Database Cluster,” ACM/IEEE Conference on Supercomputing (SC’07), Reno, NV, Nov. 10–16, p. 23.

The images are 1.7 MP, and up to 1000 pairs can be processed in this notebook. The output from the PIV function will be compared with the underlying velocity data.


## Install OpenPIV

In [1]:
# check that GPU is connect. Should say CUDA Version 1x.x
!nvidia-smi
!nvcc --version


Mon May 24 23:59:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install PyCUDA.
!pip3 install pycuda

# Install scikit-CUDA.
!pip3 install scikit-CUDA

# Install other requirements which may already be fulfilled.
!pip3 install cython imageio numpy matplotlib setuptools progressbar2

# Clone the repo.
!git clone https://github.com/ericyang125/openpiv-python-gpu.git

# Install OpenPIV extensions.
!cd openpiv-python-gpu && python3 setup.py build_ext --inplace


     |████████████████████████████████| 1.7MB 8.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627877 sha256=858c210543aeb6c1d6cda8c50cc0db1cd96f2a815f73b8b9c246a052b8fb9601
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.6-py2.py3-none-any.whl size=60643 sha256=8e2d42a05b9ba6573c923c19445aa69d71551468bd21f1690f28917e1220b960
  Stored in directory: /root/.cache/pip/wheels/8c/a6/65/447b9b4fd1d9bde84ad2fea2431a38f69f3fb573476a98ae03
Successfully built pytools
     |████████████████████████████████| 122kB 7.2MB/s 
Cloning into 'openpiv-python-gpu'...
remote: Enumerating objects: 5

## Load the data

A client authorization is required from the user to download the data from Google Drive.

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [4]:
import glob
from progressbar import progressbar

#@title Dataset
#@markdown Choose the number of image pairs to download:
start_index = 0 #@param {type:"slider", min:0, max:1000, step:1}
end_index = 50 #@param {type:"slider", min:0, max:1000, step:1}

assert start_index < end_index, 'start_index must be smaller than end_index'
print('Downloading {} files:'.format(2 * (end_index - start_index)))

# folder id of the public data
f_id = '1IuzZlz7DjjKHptILpuzRAFRUioiAiqaX'

# resolution of the images
im_shape = (512, 3217)

# get the list of files
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(f_id)}).GetList()

# sort the file list by title
file_list = sorted(file_list, key=lambda i: (i['title']))

# download the files
for i in progressbar(range(2 * start_index, 2 * end_index)):
    downloaded = drive.CreateFile({'id': file_list[i]['id']})
    downloaded.GetContentFile(file_list[i]['title'])

# make image lists
frame_a_list = sorted(glob.glob('*_a.tiff'))
frame_b_list = sorted(glob.glob('*_b.tiff'))

print('{} image pairs total have been downloaded to this VM.'.format(frame_a_list))


100% (100 of 100) |######################| Elapsed Time: 0:01:36 Time:  0:01:36


['jh_0000_a.tiff', 'jh_0001_a.tiff', 'jh_0002_a.tiff', 'jh_0003_a.tiff', 'jh_0004_a.tiff', 'jh_0005_a.tiff', 'jh_0006_a.tiff', 'jh_0007_a.tiff', 'jh_0008_a.tiff', 'jh_0009_a.tiff', 'jh_0010_a.tiff', 'jh_0011_a.tiff', 'jh_0012_a.tiff', 'jh_0013_a.tiff', 'jh_0014_a.tiff', 'jh_0015_a.tiff', 'jh_0016_a.tiff', 'jh_0017_a.tiff', 'jh_0018_a.tiff', 'jh_0019_a.tiff', 'jh_0020_a.tiff', 'jh_0021_a.tiff', 'jh_0022_a.tiff', 'jh_0023_a.tiff', 'jh_0024_a.tiff', 'jh_0025_a.tiff', 'jh_0026_a.tiff', 'jh_0027_a.tiff', 'jh_0028_a.tiff', 'jh_0029_a.tiff', 'jh_0030_a.tiff', 'jh_0031_a.tiff', 'jh_0032_a.tiff', 'jh_0033_a.tiff', 'jh_0034_a.tiff', 'jh_0035_a.tiff', 'jh_0036_a.tiff', 'jh_0037_a.tiff', 'jh_0038_a.tiff', 'jh_0039_a.tiff', 'jh_0040_a.tiff', 'jh_0041_a.tiff', 'jh_0042_a.tiff', 'jh_0043_a.tiff', 'jh_0044_a.tiff', 'jh_0045_a.tiff', 'jh_0046_a.tiff', 'jh_0047_a.tiff', 'jh_0048_a.tiff', 'jh_0049_a.tiff'] image pairs total have been downloaded to this VM.


## PIV Parameters

Here you can try changing the parameters and see how it affects the results.
Try using a *window_size_iters* = (1, 2) and *min_window_size* = 16.
Then try *window_size_iters* = (1, 2, 2) and *min_window_size* = 8

In [5]:
#@title Main Parameters

#@markdown Number of iterations performed at each window size. You can enter a tuple, where the number of tuple indexes control the number of mesh refinements. For example (1, 2, 2) would perform 1 iteration at 4 * *min_window_size*, 2 iterations at 2 * *min_window_size* and 2 iterations at *min_window_size*.
window_size_iters = (1, 2) #@param {type:"raw"}
#@markdown Length of the sides of the square deformation. Only supports multiples of 8.
min_window_size = 16 #@param [8, 16, 32] {type:"raw"}
#@markdown The ratio of overlap between two windows (between 0 and 1). Too much overlap will result in the algorithm failing.
overlap_ratio = 0.5 #@param {type:"number"}
#@markdown Time delay separating the two frames. This can be left as 1 for this tutorial.
dt = 1 #@param {type:"number"}
#@markdown 2D array of integers with values 0 for the background, 1 for the flow-field. If the center of a window is on a 0 value the velocity is set to 0.
mask = None #@param ["None"] {type:"raw"}
#@markdown Whether to deform the windows by the velocity gradient at each iteration. This improves the results if there are large gradient fields.
deform = True #@param ["True", "False"] {type:"raw"}
#@markdown Whether to smooth the intermediate fields.
smooth = True #@param ["True", "False"] {type:"raw"}
#@markdown Number of iterations per validation cycle.
nb_validation_iter = 2 #@param {type:"number"}
#@markdown Method used for validation. Only the mean velocity method is implemented for now.
validation_method = 'median_validation' #@param ['median_validation'] {type:"raw"}
#@markdown With a first window size following the 1/4 rule, the 1st iteration can be trusted.
trust_1st_iter = True #@param ["True", "False"] {type:"raw"}



In [6]:
#@title Other Parameters
#@markdown These are more advanced parameters and can be ignored by beginners.

#@markdown Tolerance of the median validation. The default value of 2 is fairly universal (Westerwheel, 1994).
median_tol = 2 #@param {type:"number"}
#@markdown One of the following methods to estimate subpixel location of the peak:\n'centroid' [replaces default if correlation map is negative],\n'gaussian' [default if correlation map is positive],\n'parabolic'.
subpixel_method = 'gaussian' #@param ['gaussian'] {type:"raw"}
#@markdown Whether the signal-to-noise ratio should be computed and returned. Setting this to False speeds up the computation significantly.
sig2noise = True #@param ["True", "False"] {type:"raw"}
#@markdown Defines the method of signal-to-noise-ratio measure. ['peak2peak' or 'peak2mean'].
sig2noise_method = 'peak2peak' #@param ['peak2peak'] {type:"raw"}
#@markdown The half size of the region around the first correlation peak to ignore for finding the second peak.\n[default: 2]. Only used if sig2noise_method==peak2peak.
width = 2 #@param {type:"number"}


## Processing

In [7]:
#@title Batch size
#@markdown Choose the number of image pairs to process at once:
batch_start_index = 0 #@param {type:"slider", min:0, max:1000, step:1}
batch_end_index = 50 #@param {type:"slider", min:0, max:1000, step:1}

#@markdown Each image pair should take O(1) s on modern hardware.

assert batch_start_index < batch_end_index, 'batch_start_index must be smaller than batch_end_index'
assert batch_start_index < len(frame_a_list), 'batch_start_index must be smaller than the number of image pairs downloaded{}'.format(len(frame_a_list))

# get the number of fields to process
batch_end_index = min(batch_end_index, len(frame_a_list))
num_fields = batch_end_index - batch_start_index

In [8]:
import os
import sys
import numpy as np
import time
import imageio as io
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
%matplotlib inline

# Add OpenPIV to the python path.
sys.path.append(os.path.join(os.getcwd(), 'openpiv-python-gpu/'))

# Import the GPU module and the tools module.
import openpiv.gpu_process as gpu_process
import openpiv.tools as tools

# The parameters are input as a dictioary for convenience
pars = {
    'window_size_iters': window_size_iters,
    'min_window_size': min_window_size,
    'overlap_ratio': overlap_ratio,
    'dt': dt,
    'mask': mask,
    'deform': deform,
    'smooth': smooth,
    'nb_validation_iter': nb_validation_iter,
    'validation_method': validation_method,
    'trust_1st_iter': trust_1st_iter,
    'median_tol': median_tol,
    'subpixel_method': subpixel_method,
    'sig2noise': sig2noise,
    'sig2noise_method': sig2noise_method,
    'width': width,
}

# Numpy arrays are created to store the velocity data.
m, n = gpu_process.get_field_shape(im_shape, min_window_size, min_window_size * overlap_ratio)  # shape of the final velocity field
s2n = ((num_fields, m, n))
u = np.empty_like(s2n)
v = np.empty_like(s2n)

# This is the main loop over the dataset
start_time = time.time()
for i in progressbar(range(batch_start_index, batch_end_index), redirect_stout=True):

    # The images are loaded
    frame_a  = io.imread(frame_a_list[i])
    frame_b  = io.imread(frame_b_list[i])

    # The velocity fields are computed.
    with redirect_stdout(None):  # suppress printing to console to keep notebook clean
        x, y, u, v, mask, s2n = gpu_process.gpu_piv_def(frame_a, frame_b, **pars)

end_time = time.time()

/usr/local/lib/python3.7/dist-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')
N/A% (0 of 50) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

ValueError: ignored

In [12]:
io.imread(frame_a_list[i]).shape

(512, 3217, 4)

## Results

In [ ]:
# The performance of the function is reported.
print('Computation time was {}'.format(end_time - start_time))
print('Time per image pair was {}'.format((end_time - start_time) / (num_fields))

In [ ]:
# The accuracy of the fields is evaluated

# graph


In [ ]:
# Run this to visualize a series of fields as quiver plots

# range = 

## Conclusions
Processing a large dataset has been shown.
The next steps are to look at masking the data and multiprocessing using one or more GPUs.

### Further steps
To learn how to dynamically mask static objects in data, see the tutorial on masking.
